In [39]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
WORKSPACE_PATH = '/home/takeyama/pywork/ipython/2016-06-20/'

mozartの環境設定

In [3]:
#p_path="/home/takeyama/pywork/ipython/2016-05-30/"

In [4]:
#cd ~/Documents/SyncRecord/cleaning-addingLABEL/

workstationの環境設定

In [5]:
cd ~/Documents/ALTIMA/20160617-103037/

/home/takeyama/Documents/ALTIMA/20160617-103037


In [44]:
'''
csv_file -> ファイル名
mode Round -> 四捨五入
     Roundup -> 切り上げ
     Rounddown -> 切り捨て
'''
def ImportCSV(csv_file,freq,SensorName,mode='Round'):
        # data dictionary 
        RawData={}   
        
        # design dataframe and import csv
        data = pd.read_csv(csv_file)
        data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        data = data[ data['Type']=='ags']
        
        # convert numpy.darray 
        # Acc Data  [0.1mG]=>[G]
        # Gyr Data  [0.01dps]=>[dps]   ...dps=degree per second
        AccX=data.AccX.values*0.0001
        AccY=data.AccY.values*0.0001
        AccZ=data.AccZ.values*0.0001
        GyrX=data.GyrX.values*0.01
        GyrY=data.GyrY.values*0.01
        GyrZ=data.GyrZ.values*0.01
        
        # regist each raw data 
        RawData['AccX'] = AccX
        RawData['AccY'] = AccY
        RawData['AccZ'] = AccZ
        RawData['GyrX'] = GyrX
        RawData['GyrY'] = GyrY
        RawData['GyrZ'] = GyrZ
        RawData['Name'] = SensorName
        
        # import time by using numpy
        time = data.Time.values #時間の列だけを抽出       
        
        if mode == 'Roundup':
            func = lambda x: int(x/freq)*freq
        elif mode == 'Rounddown':
            func = lambda x: int(x/freq)*freq
        elif mode == 'Round':
            func = lambda x: int((x+freq/2)/freq)*freq
        #ERROR
        else:
            print 'check mode and inputed word is caused error'
            return -1
        
        output = map(func,time)
        RawData['Time'] = np.array(output)

        return RawData
            

In [45]:
def CalcStartTime(array):
    MAX = min(array[0])
    
    for i in range(len(array)):
        if MAX < min(array[i]):
            MAX = min(array[i])
     
    return  MAX

In [46]:
def CalcGoalTime(array):
    MIN = max(array[0])
    
    for i in range(len(array)):
        if MIN > max(array[i]):
            MIN = max(array[i])
     
    return  MIN

**2016-06-14**  
もう１度、前処理について考える。特に、入力データの特徴、出力データの仕様を改める。  
*feacher of Precondition*
1. センサデータは{time, accx, accy, accz, gyrx, gyry, gyrz}が揃っている
2. また、センサデータはすべて整数型となっている。
3. nanデータはない

*output Data*
1. accは[0.1mG]から[G]に単位変換、gyrは[0.01dpg]は[dpg]に変換する。
2. timeはサンプリング周期で丸める。
3. Nanデータは存在する。


In [51]:
"""
    checkData に入れるものは辞書型にする
    freqは計測周期
"""
def NanPating(DicData,freq):
    import time
    start_time = time.time()
    
    # detection for hidden Nan Data
    diffNum =np.array([])
    diffIndex=np.array([])
    checkData = DicData['Time']
    width = len(checkData)
    for i in range(0,width-1):
        if ( checkData[i+1]-checkData[i] )!=freq:
            diffNum=np.append(diffNum, int(checkData[i+1]-checkData[i]) )
            diffIndex=np.append(diffIndex,i)
   
    # insert NAN data to SensorData
    # insert time_data
    def Insert(data,dI,dN,f,mode):
        StartIndex= 0
        tmp =np.array([])
        if mode =='Sensor':
            # insert NAN DATA
            adding = np.nan
            for count,l in enumerate(dI):
                tmp = np.append(tmp, data[StartIndex:int(l)])
                for i in range(0,int(dN[count]/f) ):
                    tmp = np.append(tmp,np.nan)
                StartIndex = int(l)+1
            tmp=np.append(tmp, data[StartIndex:])
        elif mode =='Time':
            # insert 
            for count,l in enumerate(dI):
                tmp = np.append(tmp, data[StartIndex:int(l)])
                for i in range(0,int(dN[count]/f) ):
                    t = int( tmp[-1]+f )
                    tmp = np.append(tmp,t)
                StartIndex = int(l)+1
            tmp=np.append(tmp, data[StartIndex:])
        else:
            print 'mode name error'
        return tmp
    Array ={}
    tmpArrayAccX=Insert(DicData['AccX'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayAccY=Insert(DicData['AccY'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayAccZ=Insert(DicData['AccZ'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayGyrX=Insert(DicData['GyrX'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayGyrY=Insert(DicData['GyrY'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayGyrZ=Insert(DicData['GyrZ'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayTime=Insert(DicData['Time'],diffIndex,diffNum,freq,mode='Time')
    Array['AccX'] = tmpArrayAccX
    Array['AccY'] = tmpArrayAccY
    Array['AccZ'] = tmpArrayAccZ
    Array['GyrX'] = tmpArrayGyrX
    Array['GyrY'] = tmpArrayGyrY
    Array['GyrZ'] = tmpArrayGyrZ
    Array['Time'] = tmpArrayTime
    Array['Name'] = DicData['Name']
    #Array=[Time:tmpArrayTime,tmpArrayAccX,tmpArrayAccY,tmpArrayAccZ,tmpArrayGyrX,tmpArrayGyrY,tmpArrayGyrZ]
    elapsed_time = time.time() -start_time
    print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
    return Array

In [52]:
def CalcSearchIndexFromTime(checkData,CompareTime):
    count = 0
    for i in range(0, len(checkData)):
        if CompareTime == checkData[i]:
            print str(CompareTime)+' is much in the index  whose number is '+str(i)
            return i
        

In [53]:
def Extraction(dic,start,goal):
    keys = dic.keys()
    output = {}
    for i in range( len(dic)):
        output[keys[i]] = dic[keys[i]][start:goal]
        if keys[i]=='Time': output[keys[i]] = output[keys[i]].astype(int)
    return output
    

In [54]:
DataCsv1= ImportCSV('14011711-20160617-103037714.csv',1,'sensor1')
DataCsv2= ImportCSV('14011712-20160617-103037714.csv',1,'sensor2')
DataCsv3= ImportCSV('14011713-20160617-103037714.csv',1,'sensor3')
DataCsv4= ImportCSV('14011714-20160617-103037714.csv',1,'sensor4')
DataCsv5= ImportCSV('11411715-20160617-103037714.csv',1,'sensor5')
DataCsv6= ImportCSV('14011716-20160617-103037714.csv',1,'sensor6')
DataCsv7= ImportCSV('14011717-20160617-103037714.csv',1,'sensor7')

In [55]:
def SearchNAN(array,freq):
    diffNum =np.array([])
    diffIndex=np.array([])
    width = len(array)
    for i in range(0,width-1):
        if ( array[i+1]-array[i] )!=freq:
            diffNum=np.append(diffNum, int(array[i+1]-array[i]) )
            diffIndex=np.append(diffIndex,i)
    print 'diffNum='+str(diffNum)
    print 'diffIndex'+str(diffIndex)

In [56]:
MemDataCsv1= ImportCSV('mem/mem-14011711-20160617-103038606.csv',1,'MemSensor1')
MemDataCsv2= ImportCSV('mem/mem-14011712-20160617-103038627.csv',1,'MemSensor2')
MemDataCsv3= ImportCSV('mem/mem-14011713-20160617-103038690.csv',1,'MemSensor3')
MemDataCsv4= ImportCSV('mem/mem-14011714-20160617-103038731.csv',1,'MemSensor4')
MemDataCsv5= ImportCSV('mem/mem-11411715-20160617-103038767.csv',1,'MemSensor5')
MemDataCsv6= ImportCSV('mem/mem-14011716-20160617-103038807.csv',1,'MemSensor6')
MemDataCsv7= ImportCSV('mem/mem-14011717-20160617-103037850.csv',1,'MemSensor7')

In [57]:
print MemDataCsv1['Time'][0]
print MemDataCsv2['Time'][0]
print MemDataCsv3['Time'][0]
print MemDataCsv4['Time'][0]
print MemDataCsv5['Time'][0]
print MemDataCsv6['Time'][0]
print MemDataCsv7['Time'][0]

37838721
37838742
37838805
37838846
37838882
37838922
37837965


In [58]:
print DataCsv1['Time'][0]
print DataCsv2['Time'][0]
print DataCsv3['Time'][0]
print DataCsv4['Time'][0]
print DataCsv5['Time'][0]
print DataCsv6['Time'][0]
print DataCsv7['Time'][0]

37838721
37838742
37838805
37838846
37838882
37838922
37837965


**2016-06-20**  
Altimaで計測して、メモリダウンロードをしたデータを生データとする。  
この生データからウィンドウ幅で切り分けて保存する。


In [59]:
MemTimeArray = [MemDataCsv1['Time'], MemDataCsv2['Time'], MemDataCsv3['Time'], MemDataCsv4['Time'],
               MemDataCsv5['Time'], MemDataCsv6['Time'], MemDataCsv7['Time']]

In [60]:
MemDataArray = [MemDataCsv1, MemDataCsv2, MemDataCsv3, MemDataCsv4, MemDataCsv5, MemDataCsv6, MemDataCsv7]

In [61]:
CalcStartTime(MemTimeArray)

37838922

In [66]:
WINDOW = 16

startTime = CalcStartTime(MemTimeArray)
goalTime = CalcGoalTime(MemTimeArray)

startIndex = np.array([])
goalIndex = np.array([])

for obj in MemTimeArray:
    print 'start'
    startIndex = np.append(startIndex, CalcSearchIndexFromTime(obj, startTime) ).astype(int)
    print 'goal'
    goalIndex = np.append(goalIndex, CalcSearchIndexFromTime(obj, goalTime) ).astype(int)

tmp={}
comMemDataArray = []
RestorePath = WORKSPACE_PATH+'/raw/'

for i,memDic in enumerate( MemDataArray ):

    tmp['AccX'] = memDic['AccX'][startIndex[i]:goalIndex[i]]
    tmp['AccY'] = memDic['AccY'][startIndex[i]:goalIndex[i]]
    tmp['AccZ'] = memDic['AccZ'][startIndex[i]:goalIndex[i]]
    tmp['GyrX'] = memDic['GyrX'][startIndex[i]:goalIndex[i]]
    tmp['GyrY'] = memDic['GyrY'][startIndex[i]:goalIndex[i]]
    tmp['GyrZ'] = memDic['GyrZ'][startIndex[i]:goalIndex[i]]
    tmp['Time'] = memDic['Time'][startIndex[i]:goalIndex[i]]
    
    tmp['Name'] = memDic['Name']
    
    if not ( os.path.exists(RestorePath) ):
        os.makedirs(RestorePath)
    
    f = open(RestorePath+tmp['Name']+'.pickle', 'wb')
    pickle.dump(tmp,f)
    comMemDataArray.append(tmp)
    
    print str( tmp['AccX'].shape )

    

start
37838922 is much in the index  whose number is 201
goal
37971170 is much in the index  whose number is 132449
start
37838922 is much in the index  whose number is 180
goal
37971170 is much in the index  whose number is 132428
start
37838922 is much in the index  whose number is 117
goal
37971170 is much in the index  whose number is 132365
start
37838922 is much in the index  whose number is 76
goal
37971170 is much in the index  whose number is 132324
start
37838922 is much in the index  whose number is 40
goal
37971170 is much in the index  whose number is 132288
start
37838922 is much in the index  whose number is 0
goal
37971170 is much in the index  whose number is 132248
start
37838922 is much in the index  whose number is 957
goal
37971170 is much in the index  whose number is 133205
(8265, 16)
(8265, 16)
(8265, 16)
(8265, 16)
(8265, 16)
(8265, 16)
(8265, 16)


だいたい上記のものをクラスとしてまとめれば良いと思う。  
問題としては柔軟性のあるクラスにしたい。メソッドを選択して前処理を行うようにする。  
すこし構成を考えついてからにする。

In [37]:
len( comMemDataArray )

7

In [92]:
comMemDataArray[0]['AccX'][0]

array([-0.0115, -0.0117, -0.0162, -0.0151, -0.0097, -0.0172, -0.0124,
       -0.016 , -0.0158, -0.0177, -0.0162, -0.0124, -0.0066, -0.0146,
       -0.0141, -0.0141])

In [124]:
"""
    dicDataは1つのセンサデータである
    また、各データ（加速度、角速度）はすべて(1234,サンプリング周期)となっていること
"""
def sliding(dicData,axis,samp):
    keys = dicData.keys()
    tmp ={}
    for i in range( 1,len(keys) ):
        tmp[keys[i]] = dicData[keys[i]][axis]

    return tmp

In [126]:
sliding(comMemDataArray[0],1,16)

{'AccX': array([-0.0102, -0.0139, -0.0119, -0.009 , -0.0134, -0.0097, -0.0127,
        -0.011 , -0.0105, -0.0102, -0.011 , -0.0068, -0.0088, -0.0039,
        -0.0036, -0.0036]),
 'AccY': array([-0.0042, -0.0027, -0.0015, -0.0013, -0.0009, -0.0027, -0.0017,
        -0.0015, -0.0022, -0.0022, -0.0034,  0.0045,  0.0052, -0.0006,
         0.0001,  0.0038]),
 'AccZ': array([ 0.962 ,  0.9711,  0.984 ,  0.9796,  0.976 ,  0.9816,  0.9889,
         0.9867,  0.9804,  0.9723,  0.9699,  0.9699,  0.9655,  0.9623,
         0.9555,  0.9557]),
 'GyrX': array([ 0.14,  0.23,  0.09,  0.  ,  0.09,  0.17,  0.  ,  0.05,  0.23,
         0.29,  0.26,  0.31,  0.14, -0.03, -0.06,  0.11]),
 'GyrY': array([-0.1 ,  0.03,  0.19,  0.17,  0.17,  0.26,  0.2 ,  0.34,  0.48,
         0.49,  0.34,  0.2 ,  0.35,  0.29,  0.13,  0.02]),
 'GyrZ': array([-0.12, -0.11, -0.23, -0.28, -0.15, -0.14, -0.09, -0.06, -0.12,
        -0.14, -0.18, -0.09, -0.02,  0.07, -0.14, -0.2 ]),
 'Time': array([37838938, 37838939, 37838940, 378389

In [131]:
"""
    precond は１つのセンサデータ
        例として、（8625,16)の配列
    windowはウィンドウ幅
"""
def CalcFFT(precond,samp):
    tmp = {}
    output = []
    for i in range( len(precond) ):
        w = sliding(precond,i,samp)
        tmp['AccX'] = np.array(fftpack.fft(w['AccX'])[1:(samp/2)+1])
        tmp['AccY'] = np.array(fftpack.fft(w['AccY'])[1:(samp/2)+1])
        tmp['AccZ'] = np.array(fftpack.fft(w['AccZ'])[1:(samp/2)+1])
        tmp['GyrX'] = np.array(fftpack.fft(w['GyrX'])[1:(samp/2)+1])
        tmp['GyrY'] = np.array(fftpack.fft(w['GyrY'])[1:(samp/2)+1])
        tmp['GyrZ'] = np.array(fftpack.fft(w['GyrZ'])[1:(samp/2)+1])
        output.append(tmp)
    return output

In [132]:
CalcFFT(comMemDataArray,16)

AttributeError: 'list' object has no attribute 'keys'